In [115]:
import math
class Number(float):
    
    def __new__(self, value):
        return float.__new__(self, value)
    
    def __init__(self, value):
        float.__init__(value)
        
    def multiply(self, value):
        return Number(self * value)
    
    def divide(self, value):
        return Number(self / value)
    
    def add(self, value):
        return Number(self + value)
    
    def subtract(self, value):
        return Number(self - value)
    
    def pow(self, value):
        return Number(self ** value)
    
    def log(self):
        return Number(math.log(self))
    
    def exp(self):
        return Number(math.exp(self))
    
    def sin(self):
        return Number(math.sin(self))
    
    def cos(self):
        return Number(math.cos(self))
    
    def tan(self):
        return Number(math.tan(self))
    
    def sinh(self):
        return Number(math.sinh(self))
    
    def cosh(self):
        return Number(math.cosh(self))
    
    def tanh(self):
        return Number(math.tanh(self))
    
    def asin(self):
        return Number(math.asin(self))
    
    def acos(self):
        return Number(math.acos(self))
    
    def atan(self):
        return Number(math.atan(self))
    
    def mod(self, value):
        return Number(self % value)
    
    def max(self, value):
        return Number(max(self, value))
    
    def min(self, value):
        return Number(min(self, value))
    
    def sqrt(self):
        return Number(math.sqrt(self))
    
    def lt(self, value):
        return Number(self < value)
    
    def gt(self, value):
        return Number(self > value)
    
    def lte(self, value):
        return Number(self <= value)
    
    def gte(self, value):
        return Number(self >= value)
    
    def where(self, test, value):
        output = 0
        if (test != 0):
            return Number(value)
        else:
            return self

In [116]:
def _air_pressure(elev, method='asce'):
    if method == 'asce':
        return elev.multiply(-0.0065).add(293).divide(293).pow(5.26).multiply(101.3)
    elif method == 'refet':
        return elev.multiply(-0.0065).add(293).divide(293).pow(9.8 / (0.0065 * 286.9)).multiply(101.3)

In [117]:
def _sat_vapor_pressure(temperature):
    return temperature.add(237.3).pow(-1).multiply(temperature).multiply(17.27).exp().multiply(0.6108)

In [118]:
def _es_slope(tmean, method='asce'):
    if method == 'refet':
        return _sat_vapor_pressure(tmean).multiply(4098.0).divide(tmean.add(237.3).pow(2))
    elif method == 'asce':
        return tmean.add(237.3).pow(-1).multiply(tmean).multiply(17.27).exp().multiply(2503.0).divide(tmean.add(237.3).pow(2))

In [119]:
def _actual_vapor_pressure(q, pair):
    return q.multiply(0.378).add(0.622).pow(-1).multiply(q).multiply(pair)

In [120]:
def _specific_humidity(ea, pair):
    return ea.multiply(-0.378).add(pair).pow(-1).multiply(ea).multiply(0.622)

In [121]:
def _vpd(es, ea):
    return es.subtract(ea).max(0)

In [122]:
def _precipitable_water(ea, pair):
    return ea.multiply(pair).multiply(0.14).add(2.1)

In [123]:
def _doy_fraction(doy):
    return doy.multiply(2.0 * math.pi / 365)

In [124]:
def _delta(doy, method='asce'):
    if method == 'asce':
        return _doy_fraction(doy).subtract(1.39).sin().multiply(0.409)
    else:
        return doy.add(284).multiply(2 * math.pi / 365).sin().multiply(23.45 * (math.pi / 180))

In [125]:
def _dr(doy):
    return _doy_fraction(doy).cos().multiply(0.033).add(1.0)

In [126]:
def _seasonal_correction(doy):
    b = doy.subtract(81).divide(364.0).multiply(2 * math.pi)
    return b.multiply(2).sin().multiply(0.1645).subtract(b.cos().multiply(0.1255)).subtract(b.sin().multiply(0.0250))

In [127]:
def _solar_time_rad(lon, time_mid, sc):
    return lon.multiply(24 / (2 * math.pi)).add(time_mid).add(sc).subtract(12)

In [128]:
def _omega(solar_time):
    omega = solar_time.multiply(2 * math.pi / 24.0)
    omega = _wrap(omega, -math.pi, math.pi)
    return omega

In [129]:
def _wrap(x, x_min, x_max):
    x_range = x_max - x_min
    return x.subtract(x_min).mod(x_range).add(x_range).mod(x_range).add(x_min)

In [130]:
def _omega_sunset(lat, delta):
    return lat.tan().multiply(-1).multiply(delta.tan()).acos()

In [131]:
def _ra_daily(lat, doy, method='asce'):
    delta = _delta(doy, method)
    omegas = _omega_sunset(lat, delta)
    theta = omegas.multiply(lat.sin()).multiply(delta.sin()).add(lat.cos().multiply(delta.cos()).multiply(omegas.sin()))

    if method == 'asce':
        ra = theta.multiply(_dr(doy)).multiply((24. / math.pi) * 4.92)
    else:
        ra = theta.multiply(_dr(doy)).multiply((24. / math.pi) * (1367 * 0.0036))
    return ra

In [132]:
def _ra_hourly(lat, lon, doy, time_mid, method='asce'):
    omega = _omega(_solar_time_rad(lon, time_mid, _seasonal_correction(doy)))
    delta = _delta(doy, method)
    omegas = _omega_sunset(lat, delta)

    omega1 = omega.subtract(math.pi / 24).max(omegas.multiply(-1)).min(omegas)
    omega2 = omega.add(math.pi / 24).max(omegas.multiply(-1)).min(omegas)
    omega1 = omega1.min(omega2)

    theta = omega2.subtract(omega1).multiply(lat.sin()).multiply(delta.sin()).add(lat.cos().multiply(delta.cos()).multiply(omega2.sin().subtract(omega1.sin())))
    if method == 'asce':
        ra = theta.multiply(_dr(doy)).multiply((12. / math.pi) * 4.92)
    else:
        ra = theta.multiply(_dr(doy)).multiply((12. / math.pi) * (1367 * 0.0036))
    return ra

In [133]:
def _rso_daily(ea, ra, pair, doy, lat):
    sin_beta_24 = _doy_fraction(doy).subtract(1.39).sin().multiply(lat).multiply(0.3).add(0.85).subtract(lat.pow(2).multiply(0.42)).sin().max(0.1)

    w = _precipitable_water(ea, pair)

    kb = w.divide(sin_beta_24).pow(0.4).multiply(-0.075).add(pair.multiply(-0.00146).divide(sin_beta_24)).exp().multiply(0.98)

    kd = kb.multiply(-0.36).add(0.35).min(kb.multiply(0.82).add(0.18))

    rso = kb.add(kd).multiply(ra)
    return rso

In [134]:
def _rso_hourly(ea, ra, pair, doy, time_mid, lat, lon, method='asce'):
    sc = _seasonal_correction(doy)
    omega = _omega(_solar_time_rad(lon, time_mid, sc))

    delta = _delta(doy, method)
    sin_beta = lat.sin().multiply(delta.sin()).add(lat.cos().multiply(delta.cos()).multiply(omega.cos()))

    w = _precipitable_water(ea, pair)

    kt = 1.0
    kb = w.divide(sin_beta.max(0.01)).pow(0.4).multiply(-0.075).add(pair.multiply(-0.00146).divide(sin_beta.max(0.01).multiply(kt))).exp().multiply(0.98)

    kd = kb.multiply(-0.36).add(0.35).min(kb.multiply(0.82).add(0.18))

    rso = kb.add(kd).multiply(ra)
    return rso

In [135]:
def _rso_simple(ra, elev):
    return ra.multiply(elev.multiply(2E-5).add(0.75))

In [136]:
def _fcd_daily(rs, rso):
    return rs.divide(rso).max(0.3).min(1.0).multiply(1.35).subtract(0.35)

In [137]:
def _fcd_hourly(rs, rso, doy, time_mid, lat, lon, method='asce'):
    sc = _seasonal_correction(doy)
    delta = _delta(doy, method)
    omega = _omega(_solar_time_rad(lon, time_mid, sc))
    beta = lat.sin().multiply(delta.sin()).add(lat.cos().multiply(delta.cos()).multiply(omega.cos())).asin()

    fcd = rs.divide(rso).max(0.3).min(1).multiply(1.35).subtract(0.35)

    fcd = fcd.max(beta.lt(0.3))

    return fcd

In [138]:
def _rnl_daily(tmax, tmin, ea, fcd):
    return tmax.add(273.16).pow(4).add(tmin.add(273.16).pow(4)).multiply(0.5).multiply(ea.sqrt().multiply(-0.14).add(0.34)).multiply(fcd).multiply(4.901E-9)

In [139]:
def _rnl_hourly(tmean, ea, fcd):
    return tmean.add(273.16).pow(4).multiply(ea.sqrt().multiply(-0.14).add(0.34)).multiply(fcd).multiply(2.042E-10)

In [140]:
def _rn(rs, rnl):
    return rnl.multiply(-1).add(rs.multiply(0.77))

In [141]:
def _wind_height_adjust(uz, zw):
    return uz.multiply(4.87).divide(zw.multiply(67.8).subtract(5.42).log())

In [142]:
def daily(tmax, tmin, ea, rs, uz, zw, elev, lat, doy, method='asce', rso_type=None, rso=None):
    tmax = Number(tmax)
    tmin = Number(tmin)
    ea = Number(ea)
    rs = Number(rs)
    uz = Number(uz)
    zw = Number(zw)
    elev = Number(elev)
    lat = Number(lat)
    doy = Number(doy)
    
    lat = lat.multiply(math.pi / 180)

    pair = _air_pressure(elev, method)

    psy = pair.multiply(0.000665)

    tmean = tmax.add(tmin).multiply(0.5)
    es_slope = _es_slope(tmean, method)

    es = _sat_vapor_pressure(tmax).add(_sat_vapor_pressure(tmin)).multiply(0.5)

    vpd = _vpd(es=es, ea=ea)

    ra = _ra_daily(lat=lat, doy=doy, method=method)

    if rso_type is None:
        if method.lower() == 'asce':
            rso = _rso_simple(ra=ra, elev=elev)
        elif method.lower() == 'refet':
            rso = _rso_daily(ea=ea, ra=ra, pair=pair, doy=doy, lat=lat)
    elif rso_type.lower() == 'simple':
        rso = _rso_simple(ra=ra, elev=elev)
    elif rso_type.lower() == 'full':
        rso = _rso_daily(ea=ea, ra=ra, pair=pair, doy=doy, lat=lat)
    elif rso_type.lower() == 'array':
        rso = rso

    fcd = _fcd_daily(rs=rs, rso=rso)

    rnl = _rnl_daily(tmax=tmax, tmin=tmin, ea=ea, fcd=fcd)

    rn = _rn(rs, rnl)

    u2 = _wind_height_adjust(uz=uz, zw=zw)
    
    cn = 1600
    cd = 0.38
    
    return tmean.add(273).pow(-1).multiply(u2).multiply(vpd).multiply(cn).multiply(psy).add(es_slope.multiply(rn).multiply(0.408)).divide(u2.multiply(cd).add(1).multiply(psy).add(es_slope))

In [143]:
def hourly(tmean, ea, rs, uz, zw, elev, lat, lon, doy, time, method='asce'):
    tmean = Number(tmean)
    ea = Number(ea)
    rs = Number(rs)
    uz = Number(uz)
    zw = Number(zw)
    elev = Number(elev)
    lat = Number(lat)
    lon = Number(lon)
    doy = Number(doy)
    time = Number(time)
    
    lat = lat.multiply(math.pi / 180)
    lon = lon.multiply(math.pi / 180)

    pair = _air_pressure(elev, method=method)

    psy = pair.multiply(0.000665)

    es = _sat_vapor_pressure(tmean)
    es_slope = _es_slope(tmean, method)

    vpd = es.subtract(ea)

    time_mid = time.add(0.5)
    ra = _ra_hourly(lat=lat, lon=lon, doy=doy, time_mid=time_mid, method=method)

    if method == 'asce':
        rso = _rso_simple(ra=ra, elev=elev)
    elif method == 'refet':
        rso = _rso_hourly(ea=ea, ra=ra, pair=pair, doy=doy, time_mid=time_mid, lat=lat, lon=lon, method=method)

    fcd = _fcd_hourly(rs=rs, rso=rso, doy=doy, time_mid=time, lat=lat, lon=lon, method=method)

    rnl = _rnl_hourly(tmean=tmean, ea=ea, fcd=fcd)

    rn = _rn(rs, rnl)

    u2 = _wind_height_adjust(uz=uz, zw=zw)
    
    cn = 66.0
    cd_day = 0.25
    g_rn_day = 0.04
    cd_night = 1.7
    g_rn_night = 0.2

    cd = rn.multiply(0).add(cd_day).where(rn.lt(0), cd_night)
    g_rn = rn.multiply(0).add(g_rn_day).where(rn.lt(0), g_rn_night)

    g = rn.multiply(g_rn)
    
    return (0.408 * es_slope * (rn - g) + (psy * cn * u2 * vpd / (tmean + 273))) / (es_slope + psy * (cd * u2 + 1))

In [110]:
tmin = (66.65 - 32) * (5.0 / 9)                          
tmax = (102.80 - 32) * (5.0 / 9)                         
tdew_c = (57.26 - 32) * (5.0 / 9)                          
ea = 0.6108 * math.exp(17.27 * tdew_c / (tdew_c + 237.3))  
rs = (674.07 * 0.041868)                                   
uz = 4.80 * 0.44704                                        
zw = 3
elev=1208.5
lat = 39.4575                                              
doy=182

In [111]:
daily(tmin=tmin, tmax=tmax, ea=ea, rs=rs, uz=uz, zw=zw, elev=elev, lat=lat, doy=doy)

10.154973542803278

In [144]:
tmean = (91.80 - 32) * (5.0 / 9)          
ea = 1.20                                   
rs = (61.16 * 0.041868)                     
uz = 3.33 * 0.44704  
zw = 3
elev = 1208.5
lat = 39.4575                               
lon = -118.77388   
doy=182
time=18

In [145]:
hourly(tmean=tmean, ea=ea, rs=rs, uz=uz, zw=zw, elev=elev, lat=lat, lon=lon, doy=doy, time=time)

0.7196147004952785